In [1]:
import numpy as np
import cv2
import dill

from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Input
from keras.optimizers import SGD, Adam, Adagrad, RMSprop
from keras.regularizers import l2, activity_l2
from keras.models import Model,Sequential
from imblearn.under_sampling import RandomUnderSampler

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def get_model(inp_shape):
    model=Sequential()
    
    model.add(Convolution2D(3, 3, 3, activation="relu",W_regularizer=l2(0.01),b_regularizer=l2(0.01),input_shape=inp_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(3, 3, 3, activation="relu",W_regularizer=l2(0.01),b_regularizer=l2(0.01),input_shape=()))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(10,activation="relu",W_regularizer=l2(0.01),b_regularizer=l2(0.01)))
    model.add(Dropout(0.25))
    model.add(Dense(1,activation="sigmoid",W_regularizer=l2(0.01),b_regularizer=l2(0.01)))
    
    #SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
    opt=RMSprop()
    model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [3]:
model=get_model((30,20,1))
model.summary()
print model.inputs

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 28, 18, 3)     30          convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 14, 9, 3)      0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 14, 9, 3)      0           maxpooling2d_1[0][0]             
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 12, 7, 3)      84          dropout_1[0][0]                  
___________________________________________________________________________________________

In [4]:
data_path='data/ser_datasets/big_set/'

X_big=np.load(data_path+'X_big.npy')
y_big=np.load(data_path+'y_big.npy')

print X_big.shape
print y_big.shape
print X_big.dtype
print y_big.dtype

(2680, 600)
(2680,)
float64
int64


In [5]:
y_big=y_big.astype(float)

X_train=X_big[:2000]
y_train=y_big[:2000]

X_test=X_big[2000:]
y_test=y_big[2000:]

X_big=[]
y_big=[]

In [6]:
img_example=X_train[0]
img_example=img_example+127
img_example=img_example.astype(np.uint8)

img_example=np.reshape(img_example,(30,20))

cv2.imshow('restored',img_example)
cv2.waitKey()

-1

In [7]:
rus = RandomUnderSampler()
X_test_resampled, y_test_resampled = rus.fit_sample(X_test, y_test)
X_train_resampled,y_train_resampled = rus.fit_sample(X_train, y_train)


X_train_resampled=np.reshape(X_train_resampled,(-1,30,20,1))
X_test_resampled=np.reshape(X_test_resampled,(-1,30,20,1))


print X_test_resampled.shape,y_test_resampled.shape
print X_train_resampled.shape,y_train_resampled.shape

(398, 30, 20, 1) (398,)
(1202, 30, 20, 1) (1202,)


In [8]:
hist=model.fit(X_train_resampled, y_train_resampled, batch_size=32, nb_epoch=50, verbose=1, validation_data=(X_test_resampled,y_test_resampled))

Train on 1202 samples, validate on 398 samples
Epoch 1/50
1202/1202 [==============================] - 0s - loss: 2.9568 - acc: 0.5358 - val_loss: 0.8440 - val_acc: 0.5352
Epoch 2/50
1202/1202 [==============================] - 0s - loss: 1.6742 - acc: 0.5116 - val_loss: 0.7567 - val_acc: 0.5050
Epoch 3/50
1202/1202 [==============================] - 0s - loss: 1.1605 - acc: 0.5441 - val_loss: 0.7035 - val_acc: 0.5553
Epoch 4/50
1202/1202 [==============================] - 0s - loss: 1.0389 - acc: 0.5516 - val_loss: 0.6852 - val_acc: 0.5879
Epoch 5/50
1202/1202 [==============================] - 0s - loss: 0.9798 - acc: 0.5616 - val_loss: 0.6651 - val_acc: 0.6005
Epoch 6/50
1202/1202 [==============================] - 0s - loss: 0.9143 - acc: 0.5774 - val_loss: 0.6573 - val_acc: 0.5854
Epoch 7/50
1202/1202 [==============================] - 0s - loss: 0.8656 - acc: 0.5749 - val_loss: 0.6293 - val_acc: 0.6131
Epoch 8/50
1202/1202 [==============================] - 0s - loss: 0.8287 - ac

In [9]:
print(hist.history.keys())

['acc', 'loss', 'val_acc', 'val_loss']


In [10]:
val_acc=hist.history['val_acc']
val_loss=hist.history['val_loss']

print len(val_acc)
print len(val_loss)

50
50


In [11]:
from matplotlib import pyplot as plt

In [12]:
plt.plot([i for i in range(len(val_acc))],val_loss)
plt.show()

In [13]:
test_res=model.evaluate(X_test_resampled,y_test_resampled)

288/398 [====================>.........] - ETA: 0s

In [14]:
print test_res

[0.13766624733581015, 0.96482412060301503]


In [15]:
model.save('neuro_models/neuro_model_0.h5')

In [16]:
cnt_zero=0
cnt_one=0
for i in y_train_resampled:
    if i==0:
        cnt_zero+=1
    elif i==1:
        cnt_one+=1
print cnt_zero,cnt_one
print y_train_resampled.shape,cnt_zero+cnt_one

601 601
(1202,) 1202
